In [1]:
from action_language_interpreter.util import solve

In [2]:
time_starts_at_0_axiom = """
time(0).
"""

In [3]:
condition_axioms = """
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, F), not pos_at(F, T).
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, -F), not neg_at(F, T).
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, unk(F)), not unk_at(F, T).
"""

In [4]:
causes_axioms = """
% E causes F if C*
pos_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, F, C), occ_at(E, T), not conditionFails_at(C, T).
neg_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, -F, C), occ_at(E, T), not conditionFails_at(C, T).
unk_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, unk(F), C), occ_at(E, T), not conditionFails_at(C, T).
"""

In [5]:
impossible_if_axiom = """
% E impossible_if C*
:- event(E), time(T), eventImpossible_if(E, C), occ_at(E, T), not conditionFails_at(C, T).
"""

In [6]:
if_axioms = """
% F if C*
pos_at(F, T) :- fluent(F), time(T), fluent_if(F, C), not conditionFails_at(C, T).
neg_at(F, T) :- fluent(F), time(T), fluent_if(-F, C), not conditionFails_at(C, T).
unk_at(F, T) :- fluent(F), time(T), fluent_if(unk(F), C), not conditionFails_at(C, T).
"""

In [7]:
inertia_axioms = """
pos_at(F, T+1) :- fluent(F), time(T), time(T+1), pos_at(F, T), not neg_at(F, T+1), not unk_at(F, T+1).
neg_at(F, T+1) :- fluent(F), time(T), time(T+1), neg_at(F, T), not pos_at(F, T+1), not unk_at(F, T+1).
unk_at(F, T+1) :- fluent(F), time(T), time(T+1), unk_at(F, T), not pos_at(F, T+1), not neg_at(F, T+1).
"""

In [8]:
consistency_axioms = """
:- fluent(F), time(T), pos_at(F, T), unk_at(F, T).
:- fluent(F), time(T), neg_at(F, T), unk_at(F, T).
"""

In [9]:
completion_g1_axiom = """
% g1
pos_at(F, 0) :- fluent(F), init(F).
neg_at(F, 0) :- fluent(F), init(-F).
"""

In [10]:
completion_g2_axiom = """
% g2
pos_at(F, 0) :- fluent(F), forced(F), default(F), not init(-F).
1 { pos_at(F, 0); neg_at(F, 0) } 1 :- fluent(F), forced(F), not default(F), not init(F), not init(-F).
"""

In [11]:
completion_g3_axiom = """
% g3
neg_at(F, 0) :- fluent(F), default(F), not pos_at(F, 0).
unk_at(F, 0) :- fluent(F), not default(F), not pos_at(F, 0), not neg_at(F, 0).
"""

In [12]:
axioms = (time_starts_at_0_axiom, condition_axioms, causes_axioms, impossible_if_axiom, if_axioms, inertia_axioms, consistency_axioms, completion_g1_axiom, completion_g2_axiom, completion_g3_axiom)

In [13]:
rules = """

% d  -> John went on a date
% m  -> John is married
% ab -> John is abnormal

time(0..1).

eventImpossible_if(d, 1).
condition_fluent(1, m).
condition_fluent(1, -ab).

"""

In [14]:
events = """

event(d).

"""

In [15]:
fluents = """

fluent(m).
fluent(ab).

"""

In [16]:
defaults = """

default(ab).

"""

In [17]:
initial = """

% empty

"""

In [18]:
action_sequence = """

occ_at(d, 0).

"""

In [19]:
query = """

query(m).

"""

In [20]:
with_defaults = """

inFluent(F) :- fluent(F).

"""

In [21]:
without_defaults = """

inFluent(F) :- fluent(F), not default(F).

"""

In [22]:
empty_inFluents = """

% empty

"""

In [23]:
forced = """

forced(F) :- inFluent(F).

"""

In [24]:
action_sequence_cross = """

split(F, T) :- occ_at(_, T), inFluent(F).

"""

$ \Pi_{AD}(I, \mathcal{F} \setminus \mathcal{D}, \aleph^{\times}) $

In [25]:
solve([
    *axioms,

    rules,

    events,
    fluents,
    defaults,
    initial,
    action_sequence,
    without_defaults,
    forced,
    action_sequence_cross,

], clingo_args=("--models", "0", "--enum-mode", "cautious"));

Answer  1: { default(ab) event(d) fluent(ab) fluent(m) forced(m) inFluent(m) time(0) time(1) conditionFails_at(1,0) conditionFails_at(1,1) condition_fluent(1,m) condition_fluent(1,-ab) eventImpossible_if(d,1) neg_at(ab,0) neg_at(ab,1) neg_at(m,0) neg_at(m,1) occ_at(d,0) split(m,0) }.
SAT 1 


<block>:3:28-35: info: atom does not occur in any rule head:
  init(F)

<block>:3:37-52: info: atom does not occur in any rule head:
  fluent_if(F,C)

<block>:3:49-73: info: atom does not occur in any rule head:
  event_causes_if(E,F,C)

<block>:3:55-63: info: atom does not occur in any rule head:
  init((-F))

<block>:4:28-36: info: atom does not occur in any rule head:
  init((-F))

<block>:4:37-53: info: atom does not occur in any rule head:
  fluent_if((-F),C)

<block>:4:49-74: info: atom does not occur in any rule head:
  event_causes_if(E,(-F),C)

<block>:4:81-88: info: atom does not occur in any rule head:
  init(F)

<block>:4:94-102: info: atom does not occur in any rule head:
  init((-F))

<block>:5:37-57: info: atom does not occur in any rule head:
  fluent_if(unk(F),C)

<block>:5:49-78: info: atom does not occur in any rule head:
  event_causes_if(E,unk(F),C)



In [26]:
# Extracted from the top answer set
R = """

neg_at(ab,0).
neg_at(ab,1).
neg_at(m,0).
neg_at(m,1).

"""

 $ I' = I \cup \{l | \{\chi(l, 0), forced(f)\} \subseteq R \land (l = f \lor l = \neg f )\} $

In [27]:
forced_initial = """

init(F) :- fluent(F), pos_at(F, 0), forced(F).
init(-F) :- fluent(F), neg_at(F, 0), forced(F).

"""

In [29]:
candidates = """

:- time(T), T = #max { T : occ_at(E, T), event(E) }, query(Q), not pos_at(Q, T+1), not neg_at(Q, T+1).

"""

For every answerset A of $ \Pi_{AD}(I', \emptyset, \aleph^?) $
such that
$ \{ \chi(\mathbb{q}, k+1), \chi(\neg\mathbb{q}, k+1) \} \cap A \neq \emptyset $

In [30]:
solve([
    *axioms,

    rules,

    events,
    fluents,
    defaults,
    initial,
    action_sequence,
    empty_inFluents,
    forced,

    R,
    forced_initial,
    query,
    candidates,

]);

Answer  1: { default(ab) event(d) fluent(ab) fluent(m) query(m) time(0) time(1) conditionFails_at(1,0) conditionFails_at(1,1) condition_fluent(1,m) condition_fluent(1,-ab) eventImpossible_if(d,1) neg_at(ab,0) neg_at(ab,1) neg_at(m,0) neg_at(m,1) occ_at(d,0) }.
SAT 1 


<block>:3:24-25: info: global variable in tuple of aggregate element:
  T

<block>:3:14-25: info: atom does not occur in any rule head:
  inFluent(F)

<block>:3:37-52: info: atom does not occur in any rule head:
  fluent_if(F,C)

<block>:3:49-73: info: atom does not occur in any rule head:
  event_causes_if(E,F,C)

<block>:4:37-53: info: atom does not occur in any rule head:
  fluent_if((-F),C)

<block>:4:49-74: info: atom does not occur in any rule head:
  event_causes_if(E,(-F),C)

<block>:5:37-57: info: atom does not occur in any rule head:
  fluent_if(unk(F),C)

<block>:5:49-78: info: atom does not occur in any rule head:
  event_causes_if(E,unk(F),C)

